<a href="https://colab.research.google.com/github/hansubkim808/spotify_net/blob/main/Spotify_Net_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spotipy

In [ ]:
import os

os.environ['SPOTIPY_CLIENT_ID']='a78e6d0e2ed240d2813c130a3e848194'
os.environ['SPOTIPY_CLIENT_SECRET']='74ceb730c73b41a98fbfdb4bda9a121d'
os.environ['SPOTIPY_REDIRECT_URI']='https://developer.spotify.com/dashboard/applications/a78e6d0e2ed240d2813c130a3e848194'

In [ ]:
import json
import spotipy
import sys
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import Model
from numpy import linalg as LA
import pandas as pd 
from statistics import mean, mode 
from IPython import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

In [ ]:
def get_audio_features(track_ids):
  af = [sp.audio_features(id) for id in track_ids]
  feature_dicts = [elem[0] for elem in af]

  audio_feature_df = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 
                           'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
  i = 0
  while i < len(feature_dicts): 
    row = feature_dicts[i]
    df_row = pd.DataFrame([row])
    audio_feature_df = audio_feature_df.append(df_row, ignore_index=True)
    i += 1

  print("Successfully formatted " + str(i) + " rows.")
  audio_feature_df = audio_feature_df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url'], axis=1) # remove irrelevant columns
  return audio_feature_df

In [ ]:
def get_artist_ids(artists):

  artist_ids = []
  artist_names = []

  for artist in artists:
    artist_results = sp.search(artist, type='artist')
    artist_name = artist_results['artists']['items'][0]['name']
    artist_id = artist_results['artists']['items'][0]['id']
    if artist_name in artist_names:
      continue
    else:
      artist_names.append(artist_name)
      artist_ids.append(artist_id)

  return artist_ids, artist_names

In [ ]:
def get_tracks_from_artist_ids(artist_ids): # takes in a list of artists 

  all_album_ids = []
  all_album_titles = []

  # Grab all albums from each artist
  for id in artist_ids:
    albums = sp.artist_albums(id)
    album_titles = [albums['items'][i]['name'] for i in range(len(albums['items']))]
    album_ids = [albums['items'][i]['id'] for i in range(len(albums['items']))]
    all_album_ids.extend(album_ids)
    all_album_titles.extend(album_titles)

  all_track_ids = []
  all_track_names = []

  # Grab all tracks in each album
  for i in range(len(all_album_ids)):
    album = all_album_ids[i]
    album_tracks = sp.album_tracks(album)
    album_track_ids = [album_tracks['items'][j]['id'] for j in range(len(album_tracks['items']))]
    album_track_names = [album_tracks['items'][k]['name'] for k in range(len(album_tracks['items']))]
    all_track_ids.extend(album_track_ids)
    all_track_names.extend(album_track_names)
  
  # Remove duplicates 
  tracks_dict = {all_track_ids[i] : all_track_names[i] for i in range(len(all_track_ids))}
  temp = {val : key for key, val in tracks_dict.items()} 
  res = {val : key for key, val in temp.items()} 
  total_track_ids = list(res.keys())
  total_track_names = list(res.values())

  return all_album_titles, all_album_ids, total_track_names, total_track_ids

In [ ]:
# List of 50 most popular artists and 50 most underrated artists.
artists = ['Post Malone', 'Duke Deuce', 'Drake', 'Lil Nas X', 'Travis Scott', 'Juice WRLD', 'DaBaby', 'Cardi B',
           'Lil Baby', 'Meek Mill', 'A Boogie Wit da Hoodie', 'Tee Grizzley', '21 Savage', 'SOB X RBE',
           'Chris Brown', 'Kodak Black', 'J. Cole', 'Young Thug', 'Lil Tecca', 'YoungBoy Never Broke Again',
           'YNW Melly', '9lokknine', 'Swae Lee', 'Eminem', 'NF', 'DJ Khaled', 'Offset', 'Lil Wayne',
           'Tyler, The Creator', 'Future', 'Blueface', 'Gunna', 'Nipsey Hussle', 'Mustard',
           'Megan Thee Stallion', 'Lil Uzi Vert', 'Polo G', '6ix9ine', 'City Girls', 'Trippie Redd',
           'Kanye West', 'Sheck Wes', 'Gucci Mane', 'Doe Boy', 'Migos', 'Roddy Ricch', 'Calboy',
           'Nicki Minaj', 'Kendrick Lamar', 'King Von', 'Jackboy', 'Shootergang Kony', '1takejay',
           'Lil Durk', 'Lil Keed', 'Lil Gotit', 'OMB Peezy', 'Foogiano', 'Big Scarr',
           'Action Pack', 'Pooh Shiesty', 'Pop Smoke', 'Fivio Foreign', '22Gz', 'K Shiday', 'Mulatto',
           '24kGoldn', 'Rich Homie Quan', 'Key Glock', 'Young Dolph', 'ABG Neal', '42 Dugg',
           'Cash Kidd', 'Bfb Da Packman', 'Teejayx6', 'Kasher Quon', 'Sada Baby', 'Drakeo the Ruler',
           'AzChike', '$tupid Young', 'Pacman da Gunman', 'Rio Da Yung Og', 'BlueBucksClan', 'Lil Zay Osama', 'Lil Poppa',
           '600 Breezy', 'Sheff G', 'Sleepy Hallow', 'Yungeen Ace', 'JayDaYoungan', 'Playboi Carti', 'UnoTheActivist',
           'Thouxanbanfauni', 'Blocboy JB', 'NLE Choppa', 'Rod Wave', 'Quando Rondo', 'Slatt Zy']


artist_ids, artist_names = get_artist_ids(artists)
album_titles, album_ids, track_names, track_ids = get_tracks_from_artist_ids(artist_ids)


In [ ]:
audio_feature_df = get_audio_features(track_ids)
audio_feature_df

Successfully formatted 8825 rows.


,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,NaN,0.3310,0.404,156253,0.645,0,4,0.1040,-3.221,0,0.0479,130.215,4,0.158
1,NaN,0.0826,0.637,150867,0.678,0,0,0.0993,-3.660,1,0.0441,132.056,4,0.287
2,NaN,0.0548,0.653,196760,0.654,0,6,0.1170,-4.239,1,0.1500,75.037,4,0.623
3,NaN,0.1410,0.671,156893,0.720,1.25e-05,0,0.3740,-3.728,1,0.0787,144.945,4,0.540
4,NaN,0.1030,0.624,221173,0.748,4.22e-05,6,0.1120,-3.452,1,0.0952,159.004,4,0.301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,NaN,0.0519,0.765,206211,0.657,0,10,0.2430,-7.622,1,0.0804,139.939,5,0.789
8821,NaN,0.0706,0.839,210077,0.599,0,8,0.1540,-9.014,0,0.0488,111.924,4,0.302
8822,NaN,0.0493,0.683,194340,0.549,0,5,0.1510,-10.143,1,0.0751,157.888,4,0.554
8823,NaN,0.0188,0.731,209784,0.568,0,1,0.2380,-6.321,0,0.0475,138.914,4,0.103


In [ ]:
audio_feature_df = audio_feature_df.drop([4839]) # row 4839 contains corrupted data 

In [ ]:
audio_feature_df

,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,NaN,0.3310,0.404,156253,0.645,0,4,0.1040,-3.221,0,0.0479,130.215,4,0.158
1,NaN,0.0826,0.637,150867,0.678,0,0,0.0993,-3.660,1,0.0441,132.056,4,0.287
2,NaN,0.0548,0.653,196760,0.654,0,6,0.1170,-4.239,1,0.1500,75.037,4,0.623
3,NaN,0.1410,0.671,156893,0.720,1.25e-05,0,0.3740,-3.728,1,0.0787,144.945,4,0.540
4,NaN,0.1030,0.624,221173,0.748,4.22e-05,6,0.1120,-3.452,1,0.0952,159.004,4,0.301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,NaN,0.0519,0.765,206211,0.657,0,10,0.2430,-7.622,1,0.0804,139.939,5,0.789
8821,NaN,0.0706,0.839,210077,0.599,0,8,0.1540,-9.014,0,0.0488,111.924,4,0.302
8822,NaN,0.0493,0.683,194340,0.549,0,5,0.1510,-10.143,1,0.0751,157.888,4,0.554
8823,NaN,0.0188,0.731,209784,0.568,0,1,0.2380,-6.321,0,0.0475,138.914,4,0.103


In [ ]:
# Save audio_feature_df as CSV
audio_feature_df.to_csv('audio_features.csv', encoding='utf-8', index=False)

In [ ]:
# Get one hot column
def one_hot_name(artists, track_ids):
  artists_dataframe = []
  for track_id in track_ids:
    name = sp.track(track_id)['artists'][0]['name']
    name_vec = np.array([1 if artist == name else 0 for artist in artists])
    one_hot_df = pd.DataFrame([[name_vec]])
    artists_dataframe.append(one_hot_df)
  if artists_dataframe is not None:
    artists_df = pd.concat(artists_dataframe, ignore_index = True, axis=0)
    artists_df.columns = ['Artist Name']
  return artists_df

In [ ]:
artists_df = one_hot_name(artists, track_ids)

In [ ]:
artists_df = artists_df.drop([4839])

In [ ]:
artists_df

,Artist Name
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8823,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
artists_df.to_csv('artists.csv', encoding='utf-8', index=True)

In [ ]:
# Get labels
def get_labels(track_ids):
  labels = [sp.track(track_id)['popularity'] for track_id in track_ids]
  labels = pd.DataFrame(labels, columns=['Popularity Index'])
  return labels

In [ ]:
y_labels = get_labels(track_ids)
y_labels

,Popularity Index
0,38
1,49
2,50
3,41
4,45
...,...
8820,32
8821,40
8822,30
8823,3


In [ ]:
y_labels = y_labels.drop([4839])
y_labels

,Popularity Index
0,38
1,49
2,50
3,41
4,45
...,...
8820,32
8821,40
8822,30
8823,3


In [ ]:
y_labels.to_csv('y_labels.csv', encoding='utf-8', index=False)

In [ ]:
# Concatenate everything together 

audio_features_df = pd.read_csv('audio_features.csv')

In [ ]:
pitches_df = pd.read_csv('pitches_total.csv')

In [ ]:
timbres_df = pd.read_csv('timbres_total.csv')

In [ ]:
all_features = pd.concat([artists_df, audio_features_df, pitches_df, timbres_df, y_labels], axis=1)

In [ ]:
all_features

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.3310,0.404,156253.0,0.645,0.000000,4.0,0.1040,-3.221,0.0,0.0479,130.215,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,52.374625,14.603359,19.317993,-9.993804,37.102729,-22.322521,8.832566,-1.847831,-1.584813,-2.577833,-10.222219,2.250311,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0826,0.637,150867.0,0.678,0.000000,0.0,0.0993,-3.660,1.0,0.0441,132.056,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,51.957452,47.972066,17.283479,-3.178558,36.985523,-17.698523,12.651432,-0.211711,-6.121857,-2.071591,-12.423403,2.447959,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0548,0.653,196760.0,0.654,0.000000,6.0,0.1170,-4.239,1.0,0.1500,75.037,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,51.232433,58.914973,14.615410,3.902505,41.423506,-16.980497,-3.030506,0.534986,-10.757905,5.101880,-14.718494,-2.096233,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1410,0.671,156893.0,0.720,0.000013,0.0,0.3740,-3.728,1.0,0.0787,144.945,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,50.233652,18.462334,-7.910042,-4.875720,26.002108,-20.819066,5.897186,-0.944734,-9.563652,-3.029483,-11.005943,5.745416,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1030,0.624,221173.0,0.748,0.000042,6.0,0.1120,-3.452,1.0,0.0952,159.004,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,50.177795,-6.796918,-4.117873,-4.121222,16.653390,-27.762054,11.601295,-3.703031,-0.888030,0.960991,-12.955758,0.220690,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0706,0.839,210077.0,0.599,0.000000,8.0,0.1540,-9.014,0.0,0.0488,111.924,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,46.996805,46.292061,15.893333,7.301978,31.095005,-7.480478,7.823466,2.378316,-16.075226,1.244836,-16.718137,-11.234359,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0493,0.683,194340.0,0.549,0.000000,5.0,0.1510,-10.143,1.0,0.0751,157.888,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,43.115753,51.878886,-4.553349,-2.268204,36.127986,-18.308030,0.466484,-1.017460,-10.734508,2.979781,-13.164195,-13.096561,40.0
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0188,0.731,209784.0,0.568,0.000000,1.0,0.2380,-6.321,0.0,0.0475,138.914,4.0,0.103,0.626991,0.639704,0.211120,0.260236,0.315421,0.188915,0.206967,0.112020,0.198373,0.140692,0.120565,0.253835,48.300213,39.128531,14.828660,6.602090,8.477929,-13.053731,20.200071,-1.140301,-16.121594,0.414891,-18.610519,-8.790546,30.0
8823,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0151,0.898,159914.0,0.511,0.000000,1.0,0.0717,-10.066,0.0,0.1110,120.030,4.0,0.390,0.363723,0.484115,0.189322,0.313000,0.289110,0.174184,0.332869,0.252478,0.449694,0.377478,0.243291,0.376472,41.060114,35.371536,-18.703566,11.908211,35.522091,-13.811962,7.231437,-8.928643,-18.866306,3.324123,-19.481358,-7.749126,3.0


In [ ]:
all_features

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.3310,0.404,156253.0,0.645,0.000000,4.0,0.1040,-3.221,0.0,0.0479,130.215,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,52.374625,14.603359,19.317993,-9.993804,37.102729,-22.322521,8.832566,-1.847831,-1.584813,-2.577833,-10.222219,2.250311,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0826,0.637,150867.0,0.678,0.000000,0.0,0.0993,-3.660,1.0,0.0441,132.056,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,51.957452,47.972066,17.283479,-3.178558,36.985523,-17.698523,12.651432,-0.211711,-6.121857,-2.071591,-12.423403,2.447959,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0548,0.653,196760.0,0.654,0.000000,6.0,0.1170,-4.239,1.0,0.1500,75.037,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,51.232433,58.914973,14.615410,3.902505,41.423506,-16.980497,-3.030506,0.534986,-10.757905,5.101880,-14.718494,-2.096233,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1410,0.671,156893.0,0.720,0.000013,0.0,0.3740,-3.728,1.0,0.0787,144.945,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,50.233652,18.462334,-7.910042,-4.875720,26.002108,-20.819066,5.897186,-0.944734,-9.563652,-3.029483,-11.005943,5.745416,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1030,0.624,221173.0,0.748,0.000042,6.0,0.1120,-3.452,1.0,0.0952,159.004,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,50.177795,-6.796918,-4.117873,-4.121222,16.653390,-27.762054,11.601295,-3.703031,-0.888030,0.960991,-12.955758,0.220690,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0706,0.839,210077.0,0.599,0.000000,8.0,0.1540,-9.014,0.0,0.0488,111.924,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,46.996805,46.292061,15.893333,7.301978,31.095005,-7.480478,7.823466,2.378316,-16.075226,1.244836,-16.718137,-11.234359,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0493,0.683,194340.0,0.549,0.000000,5.0,0.1510,-10.143,1.0,0.0751,157.888,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,43.115753,51.878886,-4.553349,-2.268204,36.127986,-18.308030,0.466484,-1.017460,-10.734508,2.979781,-13.164195,-13.096561,40.0
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0188,0.731,209784.0,0.568,0.000000,1.0,0.2380,-6.321,0.0,0.0475,138.914,4.0,0.103,0.626991,0.639704,0.211120,0.260236,0.315421,0.188915,0.206967,0.112020,0.198373,0.140692,0.120565,0.253835,48.300213,39.128531,14.828660,6.602090,8.477929,-13.053731,20.200071,-1.140301,-16.121594,0.414891,-18.610519,-8.790546,30.0
8823,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0151,0.898,159914.0,0.511,0.000000,1.0,0.0717,-10.066,0.0,0.1110,120.030,4.0,0.390,0.363723,0.484115,0.189322,0.313000,0.289110,0.174184,0.332869,0.252478,0.449694,0.377478,0.243291,0.376472,41.060114,35.371536,-18.703566,11.908211,35.522091,-13.811962,7.231437,-8.928643,-18.866306,3.324123,-19.481358,-7.749126,3.0


In [ ]:
all_features = all_features.iloc[:-1, :]
all_features

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.331000,0.404,156253.0,0.645,0.000000,4.0,0.1040,-3.221,0.0,0.0479,130.215,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,52.374625,14.603359,19.317993,-9.993804,37.102729,-22.322521,8.832566,-1.847831,-1.584813,-2.577833,-10.222219,2.250311,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.082600,0.637,150867.0,0.678,0.000000,0.0,0.0993,-3.660,1.0,0.0441,132.056,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,51.957452,47.972066,17.283479,-3.178558,36.985523,-17.698523,12.651432,-0.211711,-6.121857,-2.071591,-12.423403,2.447959,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.054800,0.653,196760.0,0.654,0.000000,6.0,0.1170,-4.239,1.0,0.1500,75.037,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,51.232433,58.914973,14.615410,3.902505,41.423506,-16.980497,-3.030506,0.534986,-10.757905,5.101880,-14.718494,-2.096233,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.141000,0.671,156893.0,0.720,0.000013,0.0,0.3740,-3.728,1.0,0.0787,144.945,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,50.233652,18.462334,-7.910042,-4.875720,26.002108,-20.819066,5.897186,-0.944734,-9.563652,-3.029483,-11.005943,5.745416,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.103000,0.624,221173.0,0.748,0.000042,6.0,0.1120,-3.452,1.0,0.0952,159.004,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,50.177795,-6.796918,-4.117873,-4.121222,16.653390,-27.762054,11.601295,-3.703031,-0.888030,0.960991,-12.955758,0.220690,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8818,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.000827,0.719,179212.0,0.623,0.000000,10.0,0.1770,-7.745,0.0,0.0483,155.105,4.0,0.599,0.394508,0.467311,0.229100,0.298683,0.253299,0.436537,0.344625,0.246663,0.234408,0.205816,0.307972,0.201680,46.396443,84.953972,-1.817812,7.748329,22.525441,-22.644441,14.221909,3.325243,-23.801723,6.736823,-18.336587,-6.889520,17.0
8819,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.051900,0.765,206211.0,0.657,0.000000,10.0,0.2430,-7.622,1.0,0.0804,139.939,5.0,0.789,0.479188,0.356685,0.358099,0.286708,0.237967,0.401666,0.229748,0.378084,0.192466,0.257882,0.399741,0.264919,47.193875,82.420550,13.640417,8.180030,44.385595,-14.312299,-1.575526,-2.526387,-16.581087,5.466996,-15.862822,-6.494747,16.0
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.070600,0.839,210077.0,0.599,0.000000,8.0,0.1540,-9.014,0.0,0.0488,111.924,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,46.996805,46.292061,15.893333,7.301978,31.095005,-7.480478,7.823466,2.378316,-16.075226,1.244836,-16.718137,-11.234359,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.049300,0.683,194340.0,0.549,0.000000,5.0,0.1510,-10.143,1.0,0.0751,157.888,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,43.115753,51.878886,-4.553349,-2.268204,36.127986,-18.308030,0.466484,-1.017460,-10.734508,2.979781,-13.164195,-13.096561,40.0


In [ ]:
all_features_modified = all_features.iloc[:-1, :]
all_features_modified

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.3310,0.404,156253.0,0.645,0.000000,4.0,0.1040,-3.221,0.0,0.0479,130.215,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,52.374625,14.603359,19.317993,-9.993804,37.102729,-22.322521,8.832566,-1.847831,-1.584813,-2.577833,-10.222219,2.250311,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0826,0.637,150867.0,0.678,0.000000,0.0,0.0993,-3.660,1.0,0.0441,132.056,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,51.957452,47.972066,17.283479,-3.178558,36.985523,-17.698523,12.651432,-0.211711,-6.121857,-2.071591,-12.423403,2.447959,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0548,0.653,196760.0,0.654,0.000000,6.0,0.1170,-4.239,1.0,0.1500,75.037,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,51.232433,58.914973,14.615410,3.902505,41.423506,-16.980497,-3.030506,0.534986,-10.757905,5.101880,-14.718494,-2.096233,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1410,0.671,156893.0,0.720,0.000013,0.0,0.3740,-3.728,1.0,0.0787,144.945,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,50.233652,18.462334,-7.910042,-4.875720,26.002108,-20.819066,5.897186,-0.944734,-9.563652,-3.029483,-11.005943,5.745416,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1030,0.624,221173.0,0.748,0.000042,6.0,0.1120,-3.452,1.0,0.0952,159.004,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,50.177795,-6.796918,-4.117873,-4.121222,16.653390,-27.762054,11.601295,-3.703031,-0.888030,0.960991,-12.955758,0.220690,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0519,0.765,206211.0,0.657,0.000000,10.0,0.2430,-7.622,1.0,0.0804,139.939,5.0,0.789,0.479188,0.356685,0.358099,0.286708,0.237967,0.401666,0.229748,0.378084,0.192466,0.257882,0.399741,0.264919,47.193875,82.420550,13.640417,8.180030,44.385595,-14.312299,-1.575526,-2.526387,-16.581087,5.466996,-15.862822,-6.494747,16.0
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0706,0.839,210077.0,0.599,0.000000,8.0,0.1540,-9.014,0.0,0.0488,111.924,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,46.996805,46.292061,15.893333,7.301978,31.095005,-7.480478,7.823466,2.378316,-16.075226,1.244836,-16.718137,-11.234359,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0493,0.683,194340.0,0.549,0.000000,5.0,0.1510,-10.143,1.0,0.0751,157.888,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,43.115753,51.878886,-4.553349,-2.268204,36.127986,-18.308030,0.466484,-1.017460,-10.734508,2.979781,-13.164195,-13.096561,40.0
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0188,0.731,209784.0,0.568,0.000000,1.0,0.2380,-6.321,0.0,0.0475,138.914,4.0,0.103,0.626991,0.639704,0.211120,0.260236,0.315421,0.188915,0.206967,0.112020,0.198373,0.140692,0.120565,0.253835,48.300213,39.128531,14.828660,6.602090,8.477929,-13.053731,20.200071,-1.140301,-16.121594,0.414891,-18.610519,-8.790546,30.0


In [ ]:
all_features_modified.fillna(0)

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.3310,0.404,156253.0,0.645,0.000000,4.0,0.1040,-3.221,0.0,0.0479,130.215,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,52.374625,14.603359,19.317993,-9.993804,37.102729,-22.322521,8.832566,-1.847831,-1.584813,-2.577833,-10.222219,2.250311,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.0826,0.637,150867.0,0.678,0.000000,0.0,0.0993,-3.660,1.0,0.0441,132.056,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,51.957452,47.972066,17.283479,-3.178558,36.985523,-17.698523,12.651432,-0.211711,-6.121857,-2.071591,-12.423403,2.447959,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.0548,0.653,196760.0,0.654,0.000000,6.0,0.1170,-4.239,1.0,0.1500,75.037,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,51.232433,58.914973,14.615410,3.902505,41.423506,-16.980497,-3.030506,0.534986,-10.757905,5.101880,-14.718494,-2.096233,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.1410,0.671,156893.0,0.720,0.000013,0.0,0.3740,-3.728,1.0,0.0787,144.945,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,50.233652,18.462334,-7.910042,-4.875720,26.002108,-20.819066,5.897186,-0.944734,-9.563652,-3.029483,-11.005943,5.745416,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.1030,0.624,221173.0,0.748,0.000042,6.0,0.1120,-3.452,1.0,0.0952,159.004,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,50.177795,-6.796918,-4.117873,-4.121222,16.653390,-27.762054,11.601295,-3.703031,-0.888030,0.960991,-12.955758,0.220690,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.0519,0.765,206211.0,0.657,0.000000,10.0,0.2430,-7.622,1.0,0.0804,139.939,5.0,0.789,0.479188,0.356685,0.358099,0.286708,0.237967,0.401666,0.229748,0.378084,0.192466,0.257882,0.399741,0.264919,47.193875,82.420550,13.640417,8.180030,44.385595,-14.312299,-1.575526,-2.526387,-16.581087,5.466996,-15.862822,-6.494747,16.0
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.0706,0.839,210077.0,0.599,0.000000,8.0,0.1540,-9.014,0.0,0.0488,111.924,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,46.996805,46.292061,15.893333,7.301978,31.095005,-7.480478,7.823466,2.378316,-16.075226,1.244836,-16.718137,-11.234359,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.0493,0.683,194340.0,0.549,0.000000,5.0,0.1510,-10.143,1.0,0.0751,157.888,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,43.115753,51.878886,-4.553349,-2.268204,36.127986,-18.308030,0.466484,-1.017460,-10.734508,2.979781,-13.164195,-13.096561,40.0
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.0188,0.731,209784.0,0.568,0.000000,1.0,0.2380,-6.321,0.0,0.0475,138.914,4.0,0.103,0.626991,0.639704,0.211120,0.260236,0.315421,0.188915,0.206967,0.112020,0.198373,0.140692,0.120565,0.253835,48.300213,39.128531,14.828660,6.602090,8.477929,-13.053731,20.200071,-1.140301,-16.121594,0.414891,-18.610519,-8.790546,30.0


In [ ]:
all_features_modified

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.3310,0.404,0.185531,0.645,0.000000,4.0,0.1040,0.938698,0.0,0.0479,0.591620,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,0.945698,0.585501,0.489807,0.268988,0.545982,0.292640,0.596374,0.666409,0.703872,0.333298,0.785758,0.527232,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0826,0.637,0.178708,0.678,0.000000,0.0,0.0993,0.931440,1.0,0.0441,0.599985,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,0.938165,0.661049,0.483199,0.319052,0.545173,0.332469,0.634909,0.686592,0.635006,0.344448,0.730692,0.531076,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0548,0.653,0.236845,0.654,0.000000,6.0,0.1170,0.921868,1.0,0.1500,0.340924,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,0.925074,0.685825,0.474534,0.371069,0.575817,0.338654,0.476667,0.695803,0.564637,0.502447,0.673277,0.442692,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1410,0.671,0.186341,0.720,0.000013,0.0,0.3740,0.930316,1.0,0.0787,0.658545,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,0.907039,0.594238,0.401376,0.306585,0.469333,0.305590,0.566754,0.677550,0.582764,0.323350,0.766152,0.595211,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1030,0.624,0.267771,0.748,0.000042,6.0,0.1120,0.934879,1.0,0.0952,0.722420,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,0.906031,0.537050,0.413692,0.312128,0.404781,0.245786,0.624313,0.643524,0.714449,0.411242,0.717374,0.487756,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0519,0.765,0.248817,0.657,0.000000,10.0,0.2430,0.865938,1.0,0.0804,0.635800,5.0,0.789,0.479188,0.356685,0.358099,0.286708,0.237967,0.401666,0.229748,0.378084,0.192466,0.257882,0.399741,0.264919,0.852152,0.739042,0.471367,0.402492,0.596270,0.361637,0.491349,0.658039,0.476249,0.510489,0.644650,0.357141,16.0
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0706,0.839,0.253715,0.599,0.000000,8.0,0.1540,0.842925,0.0,0.0488,0.508517,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,0.848594,0.657246,0.478684,0.396042,0.504499,0.420483,0.586192,0.718542,0.483927,0.417494,0.623253,0.264956,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0493,0.683,0.233779,0.549,0.000000,5.0,0.1510,0.824260,1.0,0.0751,0.717350,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,0.778516,0.669895,0.412278,0.325740,0.539251,0.327219,0.511955,0.676653,0.564992,0.455707,0.712160,0.228737,40.0
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0188,0.731,0.253343,0.568,0.000000,1.0,0.2380,0.887447,0.0,0.0475,0.631143,4.0,0.103,0.626991,0.639704,0.211120,0.260236,0.315421,0.188915,0.206967,0.112020,0.198373,0.140692,0.120565,0.253835,0.872128,0.641027,0.475226,0.390900,0.348330,0.372477,0.711081,0.675137,0.483223,0.399214,0.575912,0.312488,30.0


In [ ]:
# Normalize specified columns
from sklearn.preprocessing import MinMaxScaler
def normalize_columns(dataset):
  min_max_scaler = MinMaxScaler()
  columns_to_norm = ['loudness', 'tempo', 'duration_ms', 'Loudness', 'Brightness', 'Flatness', 'Strength of Attack', 
           'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12']
  dataset[columns_to_norm] = min_max_scaler.fit_transform(dataset[columns_to_norm])
  return dataset 

In [ ]:
input_features_normalized = normalize_columns(all_features_modified)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [ ]:
input_features_normalized

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.3310,0.404,0.185531,0.645,0.000000,4.0,0.1040,0.938698,0.0,0.0479,0.591620,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,0.945698,0.585501,0.489807,0.268988,0.545982,0.292640,0.596374,0.666409,0.703872,0.333298,0.785758,0.527232,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0826,0.637,0.178708,0.678,0.000000,0.0,0.0993,0.931440,1.0,0.0441,0.599985,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,0.938165,0.661049,0.483199,0.319052,0.545173,0.332469,0.634909,0.686592,0.635006,0.344448,0.730692,0.531076,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0548,0.653,0.236845,0.654,0.000000,6.0,0.1170,0.921868,1.0,0.1500,0.340924,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,0.925074,0.685825,0.474534,0.371069,0.575817,0.338654,0.476667,0.695803,0.564637,0.502447,0.673277,0.442692,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1410,0.671,0.186341,0.720,0.000013,0.0,0.3740,0.930316,1.0,0.0787,0.658545,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,0.907039,0.594238,0.401376,0.306585,0.469333,0.305590,0.566754,0.677550,0.582764,0.323350,0.766152,0.595211,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1030,0.624,0.267771,0.748,0.000042,6.0,0.1120,0.934879,1.0,0.0952,0.722420,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,0.906031,0.537050,0.413692,0.312128,0.404781,0.245786,0.624313,0.643524,0.714449,0.411242,0.717374,0.487756,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0519,0.765,0.248817,0.657,0.000000,10.0,0.2430,0.865938,1.0,0.0804,0.635800,5.0,0.789,0.479188,0.356685,0.358099,0.286708,0.237967,0.401666,0.229748,0.378084,0.192466,0.257882,0.399741,0.264919,0.852152,0.739042,0.471367,0.402492,0.596270,0.361637,0.491349,0.658039,0.476249,0.510489,0.644650,0.357141,16.0
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0706,0.839,0.253715,0.599,0.000000,8.0,0.1540,0.842925,0.0,0.0488,0.508517,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,0.848594,0.657246,0.478684,0.396042,0.504499,0.420483,0.586192,0.718542,0.483927,0.417494,0.623253,0.264956,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0493,0.683,0.233779,0.549,0.000000,5.0,0.1510,0.824260,1.0,0.0751,0.717350,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,0.778516,0.669895,0.412278,0.325740,0.539251,0.327219,0.511955,0.676653,0.564992,0.455707,0.712160,0.228737,40.0
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0188,0.731,0.253343,0.568,0.000000,1.0,0.2380,0.887447,0.0,0.0475,0.631143,4.0,0.103,0.626991,0.639704,0.211120,0.260236,0.315421,0.188915,0.206967,0.112020,0.198373,0.140692,0.120565,0.253835,0.872128,0.641027,0.475226,0.390900,0.348330,0.372477,0.711081,0.675137,0.483223,0.399214,0.575912,0.312488,30.0


In [ ]:
final_features_matrix = input_features_normalized
final_features_matrix

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.3310,0.404,0.185531,0.645,0.000000,4.0,0.1040,0.938698,0.0,0.0479,0.591620,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,0.945698,0.585501,0.489807,0.268988,0.545982,0.292640,0.596374,0.666409,0.703872,0.333298,0.785758,0.527232,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0826,0.637,0.178708,0.678,0.000000,0.0,0.0993,0.931440,1.0,0.0441,0.599985,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,0.938165,0.661049,0.483199,0.319052,0.545173,0.332469,0.634909,0.686592,0.635006,0.344448,0.730692,0.531076,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0548,0.653,0.236845,0.654,0.000000,6.0,0.1170,0.921868,1.0,0.1500,0.340924,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,0.925074,0.685825,0.474534,0.371069,0.575817,0.338654,0.476667,0.695803,0.564637,0.502447,0.673277,0.442692,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1410,0.671,0.186341,0.720,0.000013,0.0,0.3740,0.930316,1.0,0.0787,0.658545,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,0.907039,0.594238,0.401376,0.306585,0.469333,0.305590,0.566754,0.677550,0.582764,0.323350,0.766152,0.595211,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1030,0.624,0.267771,0.748,0.000042,6.0,0.1120,0.934879,1.0,0.0952,0.722420,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,0.906031,0.537050,0.413692,0.312128,0.404781,0.245786,0.624313,0.643524,0.714449,0.411242,0.717374,0.487756,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0519,0.765,0.248817,0.657,0.000000,10.0,0.2430,0.865938,1.0,0.0804,0.635800,5.0,0.789,0.479188,0.356685,0.358099,0.286708,0.237967,0.401666,0.229748,0.378084,0.192466,0.257882,0.399741,0.264919,0.852152,0.739042,0.471367,0.402492,0.596270,0.361637,0.491349,0.658039,0.476249,0.510489,0.644650,0.357141,16.0
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0706,0.839,0.253715,0.599,0.000000,8.0,0.1540,0.842925,0.0,0.0488,0.508517,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,0.848594,0.657246,0.478684,0.396042,0.504499,0.420483,0.586192,0.718542,0.483927,0.417494,0.623253,0.264956,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0493,0.683,0.233779,0.549,0.000000,5.0,0.1510,0.824260,1.0,0.0751,0.717350,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,0.778516,0.669895,0.412278,0.325740,0.539251,0.327219,0.511955,0.676653,0.564992,0.455707,0.712160,0.228737,40.0
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0188,0.731,0.253343,0.568,0.000000,1.0,0.2380,0.887447,0.0,0.0475,0.631143,4.0,0.103,0.626991,0.639704,0.211120,0.260236,0.315421,0.188915,0.206967,0.112020,0.198373,0.140692,0.120565,0.253835,0.872128,0.641027,0.475226,0.390900,0.348330,0.372477,0.711081,0.675137,0.483223,0.399214,0.575912,0.312488,30.0


In [ ]:
all_features_modified

,Artist Name,0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.3310,0.404,0.185531,0.645,0.000000,4.0,0.1040,0.938698,0.0,0.0479,0.591620,4.0,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,0.945698,0.585501,0.489807,0.268988,0.545982,0.292640,0.596374,0.666409,0.703872,0.333298,0.785758,0.527232,38.0
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0826,0.637,0.178708,0.678,0.000000,0.0,0.0993,0.931440,1.0,0.0441,0.599985,4.0,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,0.938165,0.661049,0.483199,0.319052,0.545173,0.332469,0.634909,0.686592,0.635006,0.344448,0.730692,0.531076,49.0
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0548,0.653,0.236845,0.654,0.000000,6.0,0.1170,0.921868,1.0,0.1500,0.340924,4.0,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,0.925074,0.685825,0.474534,0.371069,0.575817,0.338654,0.476667,0.695803,0.564637,0.502447,0.673277,0.442692,50.0
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1410,0.671,0.186341,0.720,0.000013,0.0,0.3740,0.930316,1.0,0.0787,0.658545,4.0,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,0.907039,0.594238,0.401376,0.306585,0.469333,0.305590,0.566754,0.677550,0.582764,0.323350,0.766152,0.595211,41.0
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.1030,0.624,0.267771,0.748,0.000042,6.0,0.1120,0.934879,1.0,0.0952,0.722420,4.0,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,0.906031,0.537050,0.413692,0.312128,0.404781,0.245786,0.624313,0.643524,0.714449,0.411242,0.717374,0.487756,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0519,0.765,0.248817,0.657,0.000000,10.0,0.2430,0.865938,1.0,0.0804,0.635800,5.0,0.789,0.479188,0.356685,0.358099,0.286708,0.237967,0.401666,0.229748,0.378084,0.192466,0.257882,0.399741,0.264919,0.852152,0.739042,0.471367,0.402492,0.596270,0.361637,0.491349,0.658039,0.476249,0.510489,0.644650,0.357141,16.0
8820,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0706,0.839,0.253715,0.599,0.000000,8.0,0.1540,0.842925,0.0,0.0488,0.508517,4.0,0.302,0.276600,0.522251,0.247822,0.408509,0.411277,0.320085,0.380605,0.263570,0.303668,0.188638,0.228691,0.396437,0.848594,0.657246,0.478684,0.396042,0.504499,0.420483,0.586192,0.718542,0.483927,0.417494,0.623253,0.264956,32.0
8821,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0493,0.683,0.233779,0.549,0.000000,5.0,0.1510,0.824260,1.0,0.0751,0.717350,4.0,0.554,0.540126,0.450810,0.318502,0.212732,0.245664,0.355369,0.195487,0.303526,0.213377,0.284484,0.209898,0.204560,0.778516,0.669895,0.412278,0.325740,0.539251,0.327219,0.511955,0.676653,0.564992,0.455707,0.712160,0.228737,40.0
8822,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,0.0188,0.731,0.253343,0.568,0.000000,1.0,0.2380,0.887447,0.0,0.0475,0.631143,4.0,0.103,0.626991,0.639704,0.211120,0.260236,0.315421,0.188915,0.206967,0.112020,0.198373,0.140692,0.120565,0.253835,0.872128,0.641027,0.475226,0.390900,0.348330,0.372477,0.711081,0.675137,0.483223,0.399214,0.575912,0.312488,30.0


In [ ]:
# Export to CSV
final_features_matrix.to_csv('WHOLE_DATASET.csv', encoding='utf-8')

In [ ]:
all_features.to_csv('WHOLE_DATASET_NO_NORM.csv', encoding='utf-8')

In [ ]:
whole_dataset = pd.read_csv('WHOLE_DATASET_NO_NORM (1).csv')

In [ ]:
# Normalize specified columns
from sklearn.preprocessing import MinMaxScaler
def normalize_cols(dataset):
  min_max_scaler = MinMaxScaler()
  dataset = min_max_scaler.fit_transform(dataset)
  dataset *= 2
  dataset -= 1
  return dataset 

In [ ]:
whole_dataset = normalize_cols(whole_dataset)

ValueError: ignored

In [ ]:
whole_dataset.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,0.3310,0.404,156253,0.645,0.000000,4,0.1040,-3.221,0,0.0479,130.215,4,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,52.374625,14.603359,19.317993,-9.993804,37.102729,-22.322521,8.832566,-1.847831,-1.584813,-2.577833,-10.222219,2.250311,38.0
1,0.0826,0.637,150867,0.678,0.000000,0,0.0993,-3.660,1,0.0441,132.056,4,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,51.957452,47.972066,17.283479,-3.178558,36.985523,-17.698523,12.651432,-0.211711,-6.121857,-2.071591,-12.423403,2.447959,49.0
2,0.0548,0.653,196760,0.654,0.000000,6,0.1170,-4.239,1,0.1500,75.037,4,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,51.232433,58.914973,14.615410,3.902505,41.423506,-16.980497,-3.030506,0.534986,-10.757905,5.101880,-14.718494,-2.096233,50.0
3,0.1410,0.671,156893,0.720,0.000013,0,0.3740,-3.728,1,0.0787,144.945,4,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,50.233652,18.462334,-7.910042,-4.875720,26.002108,-20.819066,5.897186,-0.944734,-9.563652,-3.029483,-11.005943,5.745416,41.0
4,0.1030,0.624,221173,0.748,0.000042,6,0.1120,-3.452,1,0.0952,159.004,4,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,50.177795,-6.796918,-4.117873,-4.121222,16.653390,-27.762054,11.601295,-3.703031,-0.888030,0.960991,-12.955758,0.220690,45.0


In [ ]:
whole_dataset.dtypes

acousticness          float64
danceability          float64
duration_ms             int64
energy                float64
instrumentalness      float64
key                     int64
liveness              float64
loudness              float64
mode                    int64
speechiness           float64
tempo                 float64
time_signature          int64
valence               float64
C                     float64
C#                    float64
D                     float64
D#                    float64
E                     float64
F                     float64
F#                    float64
G                     float64
G#                    float64
A                     float64
A#                    float64
B                     float64
Loudness              float64
Brightness            float64
Flatness              float64
Strength of Attack    float64
T5                    float64
T6                    float64
T7                    float64
T8                    float64
T9        

In [ ]:
whole_dataset['Artist Name'] = pd.Categorical(whole_dataset['Artist Name'])
whole_dataset['Artist Name'] = whole_dataset['Artist Name'].cat.codes

KeyError: ignored

In [ ]:
whole_dataset.head(50)

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,0.33100,0.404,156253,0.645,0.000000,4,0.1040,-3.221,0,0.0479,130.215,4,0.1580,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,52.374625,14.603359,19.317993,-9.993804,37.102729,-22.322521,8.832566,-1.847831,-1.584813,-2.577833,-10.222219,2.250311,38.0
1,0.08260,0.637,150867,0.678,0.000000,0,0.0993,-3.660,1,0.0441,132.056,4,0.2870,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,51.957452,47.972066,17.283479,-3.178558,36.985523,-17.698523,12.651432,-0.211711,-6.121857,-2.071591,-12.423403,2.447959,49.0
2,0.05480,0.653,196760,0.654,0.000000,6,0.1170,-4.239,1,0.1500,75.037,4,0.6230,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,51.232433,58.914973,14.615410,3.902505,41.423506,-16.980497,-3.030506,0.534986,-10.757905,5.101880,-14.718494,-2.096233,50.0
3,0.14100,0.671,156893,0.720,0.000013,0,0.3740,-3.728,1,0.0787,144.945,4,0.5400,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,50.233652,18.462334,-7.910042,-4.875720,26.002108,-20.819066,5.897186,-0.944734,-9.563652,-3.029483,-11.005943,5.745416,41.0
4,0.10300,0.624,221173,0.748,0.000042,6,0.1120,-3.452,1,0.0952,159.004,4,0.3010,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,50.177795,-6.796918,-4.117873,-4.121222,16.653390,-27.762054,11.601295,-3.703031,-0.888030,0.960991,-12.955758,0.220690,45.0
5,0.22200,0.703,215280,0.756,0.002810,0,0.0829,-3.407,1,0.0376,119.999,4,0.5360,0.685657,0.376436,0.347283,0.206694,0.360128,0.447303,0.257950,0.304767,0.233681,0.267823,0.201240,0.426808,51.887697,0.509448,-2.049847,-3.921369,26.585750,-27.635145,5.568789,-0.147402,-3.150722,-1.085416,-7.734526,6.861789,43.0
6,0.19400,0.613,245280,0.656,0.000000,0,0.2210,-2.955,1,0.0470,144.990,4,0.4430,0.483544,0.435550,0.425228,0.233289,0.333001,0.314314,0.153324,0.282055,0.141922,0.310388,0.169971,0.172018,53.152769,49.950143,47.364996,4.021323,44.181732,-16.886963,-1.837127,-6.519190,-9.620673,-1.320318,-11.321939,-5.450838,44.0
7,0.06470,0.575,218307,0.728,0.000000,7,0.1020,-5.098,1,0.0905,85.959,4,0.2210,0.508792,0.470889,0.444447,0.281277,0.294514,0.231639,0.402856,0.315638,0.210877,0.249276,0.202233,0.313606,50.734019,21.979832,9.752020,-5.477216,30.527369,-20.582188,13.897141,-3.296097,-8.322646,-0.166969,-15.593283,-4.352986,45.0
8,0.10200,0.495,229587,0.811,0.000000,1,0.1320,-2.649,0,0.0571,139.925,4,0.2410,0.409334,0.600582,0.265021,0.360166,0.390409,0.235033,0.243101,0.187187,0.400454,0.339332,0.319153,0.298597,54.130841,39.567503,21.625889,-10.348001,30.828138,-27.607112,10.688548,-0.725969,-2.681317,0.251462,-9.959401,1.141488,39.0
9,0.67500,0.748,200680,0.554,0.000002,5,0.1460,-4.039,1,0.0320,117.935,4,0.3650,0.457340,0.269188,0.246913,0.153554,0.311917,0.688073,0.223863,0.241577,0.146490,0.360660,0.227598,0.153985,49.313538,-6.280194,37.337171,8.210978,60.259426,-17.185832,7.223660,-8.080596,0.108278,-0.375335,-10.852251,1.433464,42.0


In [ ]:
import tensorflow as tf

In [ ]:
x = whole_dataset
x.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Loudness,Brightness,Flatness,Strength of Attack,T5,T6,T7,T8,T9,T10,T11,T12,Popularity Index
0,0.3310,0.404,156253,0.645,0.000000,4,0.1040,-3.221,0,0.0479,130.215,4,0.158,0.445009,0.428172,0.383228,0.250257,0.428425,0.202336,0.266594,0.286153,0.133754,0.230989,0.140266,0.322732,52.374625,14.603359,19.317993,-9.993804,37.102729,-22.322521,8.832566,-1.847831,-1.584813,-2.577833,-10.222219,2.250311,38.0
1,0.0826,0.637,150867,0.678,0.000000,0,0.0993,-3.660,1,0.0441,132.056,4,0.287,0.528395,0.427258,0.409283,0.216748,0.328459,0.339182,0.259258,0.264264,0.186665,0.357711,0.158986,0.233486,51.957452,47.972066,17.283479,-3.178558,36.985523,-17.698523,12.651432,-0.211711,-6.121857,-2.071591,-12.423403,2.447959,49.0
2,0.0548,0.653,196760,0.654,0.000000,6,0.1170,-4.239,1,0.1500,75.037,4,0.623,0.322754,0.466700,0.239175,0.328727,0.251934,0.337009,0.447116,0.297444,0.415275,0.277120,0.345526,0.261598,51.232433,58.914973,14.615410,3.902505,41.423506,-16.980497,-3.030506,0.534986,-10.757905,5.101880,-14.718494,-2.096233,50.0
3,0.1410,0.671,156893,0.720,0.000013,0,0.3740,-3.728,1,0.0787,144.945,4,0.540,0.647153,0.462689,0.323023,0.237563,0.356670,0.275443,0.225511,0.302743,0.251304,0.315230,0.197395,0.283814,50.233652,18.462334,-7.910042,-4.875720,26.002108,-20.819066,5.897186,-0.944734,-9.563652,-3.029483,-11.005943,5.745416,41.0
4,0.1030,0.624,221173,0.748,0.000042,6,0.1120,-3.452,1,0.0952,159.004,4,0.301,0.555518,0.551852,0.326906,0.331418,0.288274,0.352852,0.512763,0.274169,0.312168,0.276223,0.374591,0.408015,50.177795,-6.796918,-4.117873,-4.121222,16.653390,-27.762054,11.601295,-3.703031,-0.888030,0.960991,-12.955758,0.220690,45.0


In [ ]:
y = x.pop('Popularity Index')
x = normalize_cols(x)
x = pd.DataFrame(x)
y /= 100
dataset = tf.data.Dataset.from_tensor_slices((x.values, y.values))

In [ ]:
for feat, targ in dataset.take(15):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [-0.33333333 -0.17466803 -0.62893862  0.29648241 -1.         -0.27272727
 -0.78666667  0.87739514 -1.         -0.90010428  0.18324027  0.6
 -0.67722165 -0.23545984 -0.24903607 -0.31864392 -0.60218934 -0.22666126
 -0.71216156 -0.54590874 -0.51371499 -0.80560222 -0.59389276 -0.77910565
 -0.39618971  0.89139547  0.17100235 -0.02038618 -0.4620239   0.09196383
 -0.41472031  0.19274855  0.33281892  0.4077447  -0.33340349  0.57151639
  0.05446399], Target: 0.38
Features: [-0.83363545  0.30132789 -0.64258451  0.36281407 -1.         -1.
 -0.79630769  0.86287963  1.         -0.9080292   0.1999691   0.6
 -0.41368744 -0.04983589 -0.2510327  -0.26293884 -0.6737971  -0.44120095
 -0.41842568 -0.5613719  -0.56012884 -0.69531406 -0.33124298 -0.74036746
 -0.58017268  0.8763302   0.32209869 -0.03360154 -0.36189529  0.09034523
 -0.33506196  0.26981875  0.3731845   0.27001194 -0.31110312  0.46138431
  0.06215247], Target: 0.49
Features: [-0.88962739  0.3340143  -0.52631066  0.31457286 -1.        

In [ ]:
train_dataset = dataset.shuffle(len(x)).batch(1)

In [ ]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  from keras.optimizers import SGD
  opt = SGD(lr=0.00001, decay=1e-6, momentum=0.9)
  model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

  return model

In [ ]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

8823/8823 [==============================] - 9s 1ms/step - loss: nan - accuracy: 0.0063
Epoch 2/15
8823/8823 [==============================] - 9s 1ms/step - loss: nan - accuracy: 0.0063
Epoch 3/15
2531/8823 [=======>......................] - ETA: 6s - loss: nan - accuracy: 0.0051

KeyboardInterrupt: ignored